In [3]:
import pandas as pd
import numpy as np
import geopandas as gpd
import pyogrio
from tqdm import tqdm
tqdm.pandas()


Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


### Funcs

In [102]:
def find_segment(row):
    street_code = row['lookup_code']
    house_number = row['house_lookup']
    
    # Filter lion_gdf for segments matching streetcode1
    possible_segments = lion_gdf[lion_gdf['StreetCode'] == street_code]
    
    # Further filter based on house number and address ranges

    matching_segment = possible_segments[
        ((possible_segments['FromLeft'] <= house_number) & (possible_segments['ToLeft'] >= house_number)) |
        ((possible_segments['FromRight'] <= house_number) & (possible_segments['ToRight'] >= house_number))
    ]
    
    if not matching_segment.empty:
        return pd.Series([matching_segment.iloc[0]['geometry'], matching_segment.iloc[0]['Street']])
    else:
        return pd.Series([None, None])  # Return None if no segment matches the house number

## Read in Violations

In [60]:
data = pd.read_csv('data/Parking_Violations_Issued_-_Fiscal_Year_2024_20241011.csv') # https://data.cityofnewyork.us/City-Government/Parking-Violations-Issued-Fiscal-Year-2024/pvqr-7yc4/about_data filter Sub Division = D1
data.head()

/var/folders/4p/cnm8c74x2cscn3bny0pr_bvr0000gn/T/ipykernel_10246/454964115.py:1: DtypeWarning: Columns (18,20,36,38) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('data/Parking_Violations_Issued_-_Fiscal_Year_2024_20241011.csv') # https://data.cityofnewyork.us/City-Government/Parking-Violations-Issued-Fiscal-Year-2024/pvqr-7yc4/about_data filter Sub Division = D1


,Summons Number,Plate ID,Registration State,Plate Type,Issue Date,Violation Code,Vehicle Body Type,Vehicle Make,Issuing Agency,Street Code1,...,Vehicle Color,Unregistered Vehicle?,Vehicle Year,Meter Number,Feet From Curb,Violation Post Code,Violation Description,No Standing or Stopping Violation,Hydrant Violation,Double Parking Violation
0,1492661557,3075098,PA,PAS,07/24/2000,20,SUBN,LINCO,P,54110,...,GY,0.0,0,-,0,NaN,NaN,NaN,NaN,NaN
1,1490984033,1MCN89,NJ,PAS,08/01/2000,21,SUBN,INFIN,P,22720,...,GRY,0.0,2007,-,0,NaN,NaN,NaN,NaN,NaN
2,1492661960,BMV4559,TN,PAS,10/05/2000,98,SUBN,HONDA,P,0,...,BRN,0.0,2012,-,0,NaN,NaN,NaN,NaN,NaN
3,1493729767,JPR2146,NY,PAS,10/02/2003,21,SUBN,TOYOT,L,17390,...,BROWN,0.0,2016,-,0,NaN,NaN,NaN,NaN,NaN
4,1494022333,GTC7697,NY,PAS,02/01/2014,21,SUBN,TOYOT,P,29330,...,GY,0.0,2012,-,0,NaN,NaN,NaN,NaN,NaN


In [134]:
# convert issue date to datetime
data['Issue Date'] = pd.to_datetime(data['Issue Date'])

df = data.loc[
    data['Issue Date'].dt.year.isin([2023, 2024]),
    [
        'Issue Date',
        'Street Code1',
        'Street Code2',
        'Street Code3',
        'Violation Location',
        'Violation Precinct',
        'Violation Time',
        'Time First Observed',
        'House Number',
        'Street Name',
        'Violation County'
    ]] # trim columns


### Filter to FY24-ish

In [137]:
# many issue dates appear to be in the future or not in FY24
df['is_fy24'] = (df['Issue Date'].dt.date > pd.to_datetime('2023-06-30').date()) & (df['Issue Date'].dt.date < pd.to_datetime('2024-07-01').date())

# the issue date must not be the determining factor of what FY they fall under
# let's cast a wider net with 2 month buffer
df['is_fy24'] = (df['Issue Date'].dt.date > pd.to_datetime('2023-04-30').date()) & (df['Issue Date'].dt.date < pd.to_datetime('2024-09-01').date())

In [139]:
df['is_fy24'].value_counts()
df = df.loc[df['is_fy24']==True,:]

### Clean county names

In [142]:
# convert counties
county_dict = {
    'K':'3',
    'NY':'1',
    'Q':'4',
    'Kings':'3',
    'BX':'2',
    'Qns':'4',
    'R':'5',
    'Rich':'5',
    'QUEEN':'4',
    'Bronx':'2'
}

df['borough'] = df['Violation County'].map(county_dict)
df = df[~df['borough'].isna()] # about 1k rows are na, just drop em!

In [143]:
df['borough'].value_counts()

borough
3    937270
1    518355
4    462244
2    359890
5       128
Name: count, dtype: int64

In [144]:
# create lookup code of borough + streetcode
df['lookup_code'] = df['borough'].astype(str) + df['Street Code1'].astype(str).str.zfill(5) # streetcodes are padded to 5 digits

# encode streetcodes as ints
# Ensure street codes are integers and handle missing values
df['streetcode1'] = df['Street Code1'].astype(int)
df['streetcode2'] = df['Street Code2'].astype(int)
df['streetcode3'] = df['Street Code3'].astype(int)
df['lookup_code'] = df['lookup_code'].astype(int)

df = df.drop(columns=['Street Code1', 'Street Code2', 'Street Code3'])

In [145]:
df = df[~df['House Number'].isna()] # there's 42k records that have na house number, delete these

# the LION dataset represents dashed addresses with a zero. So 35-12 = 35012
df['House Number'] = df['House Number'].str.replace('--','-') #fix these errors
df['House Number'] = df['House Number'].str.replace(' ','0')
df['house_lookup'] = df['House Number'].str.replace('-','0').astype(int, errors='ignore')

# now coerce house lookups to ints, and drop nas
df['house_lookup'] = pd.to_numeric(df['house_lookup'], errors='coerce')
df = df.dropna(subset=['house_lookup'])


## Geocode Street Segments
via [LION Database](https://www.nyc.gov/site/planning/data-maps/open-data/dwn-lion.page)

In [68]:
# load the LION dataset
layers = pyogrio.list_layers('data/lion/lion.gdb')
print("Available layers:", layers)

Available layers: [['node' 'Point']
 ['node_stname' None]
 ['altnames' None]
 ['lion' 'MultiLineString']]


In [69]:
# Load the LION data
#lion_gdf = gpd.read_file('data/lion/lion.gdb/', layer='lion', engine='pyogrio')

# Load reprojected 4326 LION data
lion_gdf = gpd.read_file('data/lion_4326.geojson', engine='pyogrio')

lion_gdf['StreetCode'] = lion_gdf['StreetCode'].astype(int)
lion_gdf['FromLeft'] = lion_gdf['FromLeft'].astype(int)
lion_gdf['ToLeft'] = lion_gdf['ToLeft'].astype(int)
lion_gdf['FromRight'] = lion_gdf['FromRight'].astype(int)
lion_gdf['ToRight'] = lion_gdf['ToRight'].astype(int)

In [70]:
print(lion_gdf.crs)

EPSG:4326


In [104]:
df[['geometry','street']] = df.progress_apply(find_segment, axis=1)

100%|██████████| 2031717/2031717 [1:23:08<00:00, 407.25it/s]  


In [106]:
df.to_pickle('data/df.pkl')

## df Preparation

In [107]:
#read in df
df = pd.read_pickle('data/df.pkl')

In [153]:
#filter out may and june since the tickets there are minimal
df = df[df['Issue Date']>'2023-06-30']

### Total violations by month

In [154]:
df['Month-Year'] = df['Issue Date'].dt.to_period('M')
df_violations_by_month = df.groupby('Month-Year').size().reset_index(name='Violations')

In [155]:
df_violations_by_month.to_pickle('data/by_month.pkl')
df_violations_by_month.head()

,Month-Year,Violations
0,2023-07,166205
1,2023-08,174995
2,2023-09,125649
3,2023-10,163628
4,2023-11,144358


### Total violations by hour

In [113]:
def convert_violation_time(time_str):
    # Handle NaN values
    if pd.isna(time_str):
        return None
    
    try:
        # Check if time string ends with 'A' or 'P' (AM/PM format)
        if time_str[-1] in ['A', 'P']:
            period = time_str[-1]
            time_without_suffix = time_str[:-1]
            # Insert colon between hours and minutes
            formatted_time = time_without_suffix[:-2] + ':' + time_without_suffix[-2:]
            if period == 'A':
                if formatted_time.startswith('12'):
                    formatted_time = '00' + formatted_time[2:]  # Convert midnight (12:00 AM) to 00:00
                return pd.to_datetime(formatted_time, format='%H:%M').time()  # 12-hour AM format
            elif period == 'P':
                return pd.to_datetime(formatted_time + ' PM', format='%I:%M %p').time()  # 12-hour PM format
        else:
            # Assume it's in 24-hour format (HH:MM)
            return pd.to_datetime(time_str, format='%H:%M').time()  # 24-hour format

    except (ValueError, TypeError):
        # If the format doesn't match or there's another issue, return None
        return None

In [114]:
# convert violation time to times
df['time'] = df['Violation Time'].apply(convert_violation_time)
df['hour'] = df['time'].apply(lambda t: t.hour if pd.notnull(t) else None) # create hour col for aggregation
df_violations_by_hour = df[['hour']].groupby('hour').size().reset_index(name='Violations')

from datetime import time
df_violations_by_hour['time'] = df_violations_by_hour['hour'].apply(lambda h: time(int(h), 0)) # re-add time column to output

In [115]:
df_violations_by_hour.to_pickle('data/by_hour.pkl')
df_violations_by_hour.head()

,hour,Violations,time
0,0.0,8226,00:00:00
1,1.0,18226,01:00:00
2,2.0,6423,02:00:00
3,3.0,5774,03:00:00
4,4.0,2699,04:00:00


### Heatmap data

In [117]:
# Pre-process heatmap data
gdf = gpd.GeoDataFrame(df, geometry='geometry')
gdf = gdf[gdf.is_valid] # we have about 45k streets that could not be located, drop them

gdf['lat'] = gdf['geometry'].centroid.y
gdf['long'] = gdf['geometry'].centroid.x

df_heatmap_data = gdf[['lat', 'long', 'street']].groupby(['lat','long','street']).size().reset_index(name='violations')
df_heatmap_data['log_violations'] = np.log(df_heatmap_data['violations'])


In [118]:
df_heatmap_data.to_pickle('data/heatmap_df.pkl')
df_heatmap_data.head()

,lat,long,street,violations,log_violations
0,40.529218,-74.158486,STRAWBERRY LANE,1,0.0
1,40.544681,-74.219145,BLOOMINGDALE ROAD,1,0.0
2,40.552256,-74.174452,LOVELACE AVENUE,1,0.0
3,40.553316,-74.194823,HAMPTON GREEN,1,0.0
4,40.558953,-74.124045,MALONE AVENUE,1,0.0


In [36]:
# Pre-process heatmap data
gdf = gpd.GeoDataFrame(df, geometry='geometry')
print(len(gdf[gdf.is_empty]))
gdf1 = gdf.set_crs(epsg=2263) #reprojec
print(len(gdf1[gdf1.is_empty]))
gdf1 = gdf1[gdf1.is_valid] # we have about 45k streets that could not be located, drop the,
print(len(gdf1[gdf1.is_empty]))
gdf2 = gdf1.to_crs(epsg=4326)


0
0
0


ProjError: x, y, z, and time must be same size

In [43]:
gdf.to_file("data/parking_violations.geojson", driver="GeoJSON")

TypeError: Cannot interpret 'period[M]' as a data type

,Issue Date,Violation Location,Violation Precinct,Violation Time,Time First Observed,House Number,Street Name,Violation County,is_fy24,borough,lookup_code,streetcode1,streetcode2,streetcode3,house_lookup,geometry,Month-Year,time,hour
610,2023-07-01,5.0,5,0730P,0730P,215,BOWERY,NY,True,1,113460,13460,29890,29290,215.0,POLYGON EMPTY,2023-07,19:30:00,19.0
611,2023-07-01,102.0,102,0816A,00000,84-11,JAMAICA AVENUE,Qns,True,4,451090,51090,45590,17340,84011.0,POLYGON EMPTY,2023-07,08:16:00,8.0
612,2023-07-01,102.0,102,0809A,00000,84-45,JAMAICA AVENUE,Qns,True,4,451090,51090,45590,17340,84045.0,POLYGON EMPTY,2023-07,08:09:00,8.0
613,2023-07-01,102.0,102,0810A,00000,84-45,JAMAICA AVENUE,Qns,True,4,451090,51090,45590,17340,84045.0,POLYGON EMPTY,2023-07,08:10:00,8.0
615,2023-07-01,105.0,105,0745A,00000,216-07,JAMAICA AVENUE,Qns,True,4,451090,51090,26690,26740,216007.0,POLYGON EMPTY,2023-07,07:45:00,7.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2279053,2024-08-30,76.0,76,0154A,00000,640,COLUMBIA STREET,Kings,True,3,328430,28430,18950,308,640.0,POLYGON EMPTY,2024-08,01:54:00,1.0
2279054,2024-08-30,76.0,76,0156A,00000,640,COLUMBIA STREET,Kings,True,3,328430,28430,18950,308,640.0,POLYGON EMPTY,2024-08,01:56:00,1.0
2279055,2024-08-30,76.0,76,0158A,00000,556,COLUMBIA STREET,Kings,True,3,328430,28430,15935,77250,556.0,POLYGON EMPTY,2024-08,01:58:00,1.0
2279056,2024-08-30,76.0,76,0159A,00000,556,COLUMBIA STREET,Kings,True,3,328430,28430,15935,77250,556.0,POLYGON EMPTY,2024-08,01:59:00,1.0
